In [ ]:
import os
import dotenv
from termcolor import colored
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from llm import LLM

dotenv.load_dotenv()
gpt35t = LLM("gpt-3.5-turbo")
gpt4 = LLM("gpt-4")

## 1. Make quadratic problems

In [ ]:
from make_quadratic_problems import make_quadratic_problem

dotenv.load_dotenv()
llm = LLM("gpt-3.5-turbo")

for max in range(5, 50, 5):
    for factorable in [True, False]:
        filename = f'data/quadratic_problems/quadratic_problems_{max}_{factorable}.jsonl'
        if not os.path.exists(filename):
            print(f"Making problems ({max=}, {factorable=}), saving to {filename}", 'blue')
            with open(f'data/quadratic_problems/quadratic_problems_{max}_{factorable}.jsonl', 'w') as outfile:
                for _ in range(100):
                    outfile.write(make_quadratic_problem(max, factorable) + '\n')
        else:
            print(colored(f"Skipping making problems ({max=}, {factorable=}) because {filename} already exists", 'blue'))

## 2. Make quadratic-solving contexts with GPT-3.5-turbo and GPT-4

In [ ]:
from make_quadratic_contexts import solve_quadratic_problems

# prompt is an unprincipled DoF here.
# pros: makes expected switching behavior clear
# cons: model is still in "assisstant mode", not "trying to solve the problem as efficiently as possible mode"
prompt = "Please find the roots of the quadratic equation {equation}. Start by trying to factor the equation. If you can't factor it, then use the quadratic formula. If you factor the equation successfully, do not use the quadratic formula."

# remember to force feed the model so it starts by attempting factoring! Otherwise switch rate will be too low.
false_start = "First, I'll try solving this equation by factoring."

for llm in [gpt35t, gpt4]:
    for problem_filename in os.listdir('data/quadratic_problems'):
        outfile = f'data/quadratic_contexts_{llm.model_name}/' + str.replace(problem_filename, 'problem', 'context')
        if not os.path.exists(outfile):
            print(f"Solving problems in {problem_filename}, writing to {outfile}")
            solve_quadratic_problems(problem_filename, prompt, false_start, llm)
        else:
            print(colored(f"Skipping solving problems in {problem_filename} because {outfile} already exists", 'blue'))

In [ ]:

for dir in ['data/quadratic_contexts_gpt-3.5-turbo/', 'data/quadratic_contexts_gpt-4/']:
    for filename in os.listdir(dir):
        split_name = filename.split('_')
        difficulty = split_name[-2]
        is_factorizable = split_name[-1].split('.')[0]
        with open(dir+'/'+filename, 'r') as f:
            v = np.mean(['formula' in line for line in f.readlines()])
            plt.plot(int(difficulty), v, 'ro' if is_factorizable == 'True' else 'bo')
    plt.title(dir)
    plt.xlabel('difficulty parameter')
    plt.ylabel('proportion of problems solved with formula')
    plt.ylim(0, 1)
    red_dot, = plt.plot([], [], 'ro', label='factorizable')
    blue_dot, = plt.plot([], [], 'bo', label='not factorizable')
    plt.legend(handles=[red_dot, blue_dot])
    plt.show()
            

## 3. Split contexts into prefixes, judge switching, and load into a dataframe

In [ ]:
from new_switch_utils import make_non_decreasing, original_4

pref_res = 50

In [ ]:
if not os.path.exists('data/passages_35t.csv'):
    print("Making passages_35t")
    passages_35t = []
    for filename in tqdm(os.listdir('data/quadratic_contexts_gpt-3.5-turbo')):
        # filename has format quadratic_contexts_[difficulty]_[is_factorizable].jsonl
        # Each jsonl file is a list of objects, one json object per line. each object has 'equation' and 'context' field
        split_name = filename.split('_')
        difficulty = split_name[-2]
        is_factorizable = split_name[-1].split('.')[0]

        with open('data/quadratic_contexts_gpt-3.5-turbo/'+filename, 'r') as f:
            for line in f.readlines():
                line_contents = json.loads(line)
                prefixes = [line_contents['context'][:pref_end] for pref_end in range(pref_res, len(line), pref_res)]
                measured_switching = [original_4(line_contents['context'], prefix) for prefix in prefixes]
                processed_switching = make_non_decreasing(measured_switching)
                switch_index = processed_switching.index(1) if 1 in processed_switching else len(processed_switching[0])
                for i, prefix in enumerate(prefixes):
                    passages_35t.append({
                        'prefix': prefix,
                        'difficulty': difficulty,
                        'is_factorizable': 'Yes' if is_factorizable == 'True' else 'No',
                        'context': line_contents['context'],
                        'equation': line_contents['equation'],
                        'measured_switching': measured_switching,
                        'processed_switching': processed_switching,
                        'index': i,
                        'switch_index': switch_index
                    })
    passages_35t = pd.DataFrame(passages_35t)
    passages_35t.to_csv('data/passages_35t.csv')
else:
    print(colored("Skipping making passages_35t because it already exists", 'blue'))
    passages_35t = pd.read_csv('data/passages_35t.csv')
    passages_35t['measured_switching'] = passages_35t['measured_switching'].apply(lambda x: [int(x) for x in x.strip("[]").split('], [')[0].split(", ")])
    passages_35t['processed_switching'] = passages_35t['processed_switching'].apply(lambda x: [int(x) for x in x.strip("[]").split('], [')[0].split(", ")])
    passages_35t['switch_index'] = passages_35t['processed_switching'].apply(lambda x: x.index(1) if 1 in x else len(x))

if not os.path.exists('data/passages_4.csv'):
    print("Making passages_4")
    passages_4 = []
    for filename in tqdm(os.listdir('data/quadratic_contexts_gpt-4')):
        # filename has format quadratic_contexts_[difficulty]_[is_factorizable].jsonl
        # Each jsonl file is a list of objects, one json object per line. each object has 'equation' and 'context' field
        split_name = filename.split('_')
        difficulty = split_name[-2]
        is_factorizable = split_name[-1].split('.')[0]

        with open('data/quadratic_contexts_gpt-4/'+filename, 'r') as f:
            for line in f.readlines():
                line_contents = json.loads(line)
                prefixes = [line_contents['context'][:pref_end] for pref_end in range(pref_res, len(line), pref_res)]
                measured_switching = [original_4(line_contents['context'], prefix) for prefix in prefixes]
                processed_switching = make_non_decreasing(measured_switching)
                switch_index = processed_switching.index(1) if 1 in processed_switching else len(processed_switching[0])
                for i, prefix in enumerate(prefixes):
                    passages_4.append({
                        'prefix': prefix,
                        'difficulty': difficulty,
                        'is_factorizable': 'Yes' if is_factorizable == 'True' else 'No',
                        'context': line_contents['context'],
                        'equation': line_contents['equation'],
                        'measured_switching': measured_switching,
                        'processed_switching': processed_switching,
                        'index': i,
                        'switch_index': switch_index
                    })
    passages_4 = pd.DataFrame(passages_4)
    passages_4.to_csv('data/passages_4.csv')
else:
    print(colored("Skipping making passages_4 because it already exists", 'blue'))
    passages_4 = pd.read_csv('data/passages_4.csv')
    passages_4['measured_switching'] = passages_4['measured_switching'].apply(lambda x: [int(x) for x in x.strip("[]").split('], [')[0].split(", ")])
    passages_4['processed_switching'] = passages_4['processed_switching'].apply(lambda x: [int(x) for x in x.strip("[]").split('], [')[0].split(", ")])
    passages_4['switch_index'] = passages_4['processed_switching'].apply(lambda x: x.index(1) if 1 in x else len(x))

In [ ]:
passages_4['processed_switching'].value_counts()

In [ ]:
plt.hist(passages_4['switch_index'])
plt.show()
plt.hist(passages_35t['switch_index'])
plt.show()

## 4. Assess 1t and COT CPC for prefixes

In [ ]:
import concurrent.futures as futures
from solver import perform_one_token_cpc, perform_cot_cpc

def process_row(llm, row):
    row['cpc_one_token'] = perform_one_token_cpc(llm, row['prefix'])
    cot, answer = perform_cot_cpc(llm, row['prefix'])
    row['cpc_cot_thoughts'] = cot
    row['cpc_cot_answer'] = answer
    return row

def experiment2(llm, passages: pd.DataFrame) -> pd.DataFrame:
    """
    Processes the rows in the given dataframe which have not already been processed.

    :dataframe: a dataframe with at least columns 'equation', 'difficulty', 'is_factorizable', and 'context'.
    Some rows may already have been processed, in which case they will have 'one_token_cpc_result' and 'cot_cpc_result'
    columns, or else a value in the 'error' column.
    :return: dataframe with the results of processing all rows in the given `passages` dataframe
    """
    # turn passages into a rows dict including index (id, numerical)
    rows = []
    try:
        with futures.ThreadPoolExecutor() as executor:
            row_futures = [executor.submit(process_row, llm, row) for row in passages.to_dict(orient='records')]
            for future in tqdm(futures.as_completed(row_futures), total=len(row_futures)):
                if future.exception():
                    print(f'Error processing row: {type(future.exception())} {str(future.exception())}')
                    continue
                rows.append(future.result())
    except Exception as e:
        print(f'Error processing rows: {type(e)} {str(e)}')
    finally:
        return pd.DataFrame(rows)

In [ ]:
if not os.path.exists('data/full_data_35t.csv'):
    print("Making full_data_35t")
    full_data_35t = experiment2(gpt35t, passages_35t)
    full_data_35t.to_csv('data/full_data_35t.csv')
else:
    print(colored("Skipping making full_data_35t because it already exists", 'blue'))
    full_data_35t = pd.read_csv('data/full_data_35t.csv')

if not os.path.exists('data/full_data_4.csv'):
    print("Making full_data_4")
    full_data_4 = experiment2(gpt4, passages_4)
    full_data_4.to_csv('data/full_data_4.csv')
else:
    print(colored("Skipping making full_data_4 because it already exists", 'blue'))
    full_data_4 = pd.read_csv('data/full_data_4.csv')

## 5. Save results and analyze

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import sem

# Convert columns to numeric
full_data_35t['dist_to_switch'] = full_data_35t['index'] - full_data_35t['switch_index']
full_data_35t['cpc_one_token_numeric'] = full_data_35t['cpc_one_token'].map({'Yes': 1, 'No': 0})
full_data_35t['cpc_cot_answer_numeric'] = full_data_35t['cpc_cot_answer'].map({'Yes': 1, 'No': 0})

# Group the data and calculate mean and sem
grouped = full_data_35t.groupby('dist_to_switch')[['cpc_one_token_numeric']].agg(['mean', 'count', sem]).reset_index()
grouped_cot = full_data_35t.groupby('dist_to_switch')[['cpc_cot_answer_numeric']].agg(['mean', 'count', sem]).reset_index()

# Calculate confidence intervals
grouped['cpc_one_token_numeric_ci_low'] = grouped['cpc_one_token_numeric', 'mean'] - 1.96 * grouped['cpc_one_token_numeric', 'sem']# / np.sqrt(grouped['cpc_one_token_numeric', 'count'])
grouped['cpc_one_token_numeric_ci_high'] = grouped['cpc_one_token_numeric', 'mean'] + 1.96 * grouped['cpc_one_token_numeric', 'sem']# / np.sqrt(grouped['cpc_one_token_numeric', 'count'])

grouped_cot['cpc_cot_ci_low'] = grouped_cot['cpc_cot_answer_numeric', 'mean'] - 1.96 * grouped_cot['cpc_cot_answer_numeric', 'sem']# / np.sqrt(grouped_cot['cpc_cot_answer_numeric', 'count'])
grouped_cot['cpc_cot_ci_high'] = grouped_cot['cpc_cot_answer_numeric', 'mean'] + 1.96 * grouped_cot['cpc_cot_answer_numeric', 'sem']# / np.sqrt(grouped_cot['cpc_cot_answer_numeric', 'count'])

# Plot the data
plt.figure(figsize=(10, 6))
plt.errorbar(grouped['dist_to_switch'], grouped['cpc_one_token_numeric', 'mean'], yerr=[grouped['cpc_one_token_numeric', 'mean'] - grouped['cpc_one_token_numeric_ci_low'], grouped['cpc_one_token_numeric_ci_high'] - grouped['cpc_one_token_numeric', 'mean']], fmt='o', capsize=5, label='CPC One Token')
plt.errorbar(grouped_cot['dist_to_switch'], grouped_cot['cpc_cot_answer_numeric', 'mean'], yerr=[grouped_cot['cpc_cot_answer_numeric', 'mean'] - grouped_cot['cpc_cot_ci_low'], grouped_cot['cpc_cot_ci_high'] - grouped_cot['cpc_cot_answer_numeric', 'mean']], fmt='o', capsize=5, color='r', label='CPC COT')
plt.vlines(0, 0, 1, linestyles='dashed', colors='gray')
plt.xlabel('Distance to Switch')
plt.ylabel('Average Value')
plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import sem

# Convert columns to numeric
full_data_4['dist_to_switch'] = full_data_4['index'] - full_data_4['switch_index']
full_data_4['cpc_one_token_numeric'] = full_data_4['cpc_one_token'].map({'Yes': 1, 'No': 0})
full_data_4['cpc_cot_answer_numeric'] = full_data_4['cpc_cot_answer'].map({'Yes': 1, 'No': 0})

# Group the data and calculate mean and sem
grouped = full_data_4.groupby('dist_to_switch')[['cpc_one_token_numeric']].agg(['mean', 'count', sem]).reset_index()
grouped_cot = full_data_4.groupby('dist_to_switch')[['cpc_cot_answer_numeric']].agg(['mean', 'count', sem]).reset_index()

# Calculate confidence intervals
grouped['cpc_one_token_numeric_ci_low'] = grouped['cpc_one_token_numeric', 'mean'] - 1.96 * grouped['cpc_one_token_numeric', 'sem']# / np.sqrt(grouped['cpc_one_token_numeric', 'count'])
grouped['cpc_one_token_numeric_ci_high'] = grouped['cpc_one_token_numeric', 'mean'] + 1.96 * grouped['cpc_one_token_numeric', 'sem']# / np.sqrt(grouped['cpc_one_token_numeric', 'count'])

grouped_cot['cpc_cot_ci_low'] = grouped_cot['cpc_cot_answer_numeric', 'mean'] - 1.96 * grouped_cot['cpc_cot_answer_numeric', 'sem']# / np.sqrt(grouped_cot['cpc_cot_answer_numeric', 'count'])
grouped_cot['cpc_cot_ci_high'] = grouped_cot['cpc_cot_answer_numeric', 'mean'] + 1.96 * grouped_cot['cpc_cot_answer_numeric', 'sem']# / np.sqrt(grouped_cot['cpc_cot_answer_numeric', 'count'])

# Plot the data
plt.figure(figsize=(10, 6))
plt.errorbar(grouped['dist_to_switch'], grouped['cpc_one_token_numeric', 'mean'], yerr=[grouped['cpc_one_token_numeric', 'mean'] - grouped['cpc_one_token_numeric_ci_low'], grouped['cpc_one_token_numeric_ci_high'] - grouped['cpc_one_token_numeric', 'mean']], fmt='o', capsize=5, label='CPC One Token')
plt.errorbar(grouped_cot['dist_to_switch'], grouped_cot['cpc_cot_answer_numeric', 'mean'], yerr=[grouped_cot['cpc_cot_answer_numeric', 'mean'] - grouped_cot['cpc_cot_ci_low'], grouped_cot['cpc_cot_ci_high'] - grouped_cot['cpc_cot_answer_numeric', 'mean']], fmt='o', capsize=5, color='r', label='CPC COT')
plt.vlines(0, 0, 1, linestyles='dashed', colors='gray')
plt.xlabel('Distance to Switch')
plt.ylabel('Average Value')
plt.ylim(0, 1)
plt.legend()
plt.show()